# Assignment - kNN implementation with Spark 

In this assignment you will implement the kNN algorithm in Spark, and use your implementation to classify text documents. “Classification”
is the task of labeling documents based upon their contents. You will be asked to perform 4 tasks, covering data preparation, feature extraction, and classification.

## Collaboration

If you worked with a partner on this assignment. Please indicate with whom you worked (names and netIds of all team members).

In [ ]:
Michael Kelley, mpk7
Jenny Bechtold, jlb14

## Data

The dataset you will be using in this homework is the same as the dataset you used in the Spark introduction lab. That is the widely-used “20 newsgroups” dataset. A newsgroup post is like an old-school blog post, and this dataset has 19,997 such posts from 20 different categories, according to where the blog post was made. 

The 20 categories are listed in the file `news_categories.txt`. 

For each document, the category name can be extracted from the id of the document. For example, 
* the document with the id `20_newsgroups/comp.graphics/37261` is from the `comp.graphics` category, 
* the document with the id `20_newsgroups/sci.med/59082` is from the `sci.med` category. 

The data file has one line per document of text. It can be accessed at:

`s3://s21risamyersbucket/lab/20_news_same_line.txt`

We have also provided a small subset of the data in the file `20_news_same_line_random_sample.txt`, so that you can debug your Spark code on a small dataset, before running it on the entire dataset. 

# Install Java, Spark, and Findspark
This installs Apache Spark, Java, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.
[link text](https://)

In [6]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.2-bin-hadoop2.7.tgz
!pip install -q findspark

/bin/bash: apt-get: command not found
/bin/bash: wget: command not found
tar: Error opening archive: Failed to open 'spark-3.0.2-bin-hadoop2.7.tgz'


## Set Environment Variables

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"
os.environ["PYTHONHASHSEED"] = "0"

## Start a pySpark Session

In [5]:
import findspark
findspark.init()
import pyspark
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate()


Exception: Unable to find py4j, your SPARK_HOME may not be configured correctly

## Load useful modules

In [4]:
import re
import numpy as np

## (20 pts) Task 1 - compute "bag of words" for each document 

For task 1, we want to extract "bag of words" features for documents. 

The first part of this task is the same as what you've already implemented in lab. We need a dictionary, as an RDD, that includes the 20,000 most frequent words
in the training corpus. The result of such an RDD must be in this format:
`
[('mostcommonword', 0),
 ('nextmostcommonword', 1),
 ...]
`

**NOTE**: There aren’t 20,000 unique words in the small dataset (`20_news_same_line_random_sample.txt`). Use only the top 50 words when working with this file.

For this part, we provided our code, so that you only need to run it, to create this dictionary, named `refDict`, as an RDD. This `refDict` RDD will be our reference dictionary of words. The words in `refDict` will be our reference words for which we will compute "bag of words" and "TF-IDF" features for our training corpus and finally for the test documents.

**Provided code to create the reference dictionary of words.**

Run the code cells below to create the `refDict` RDD.

In [5]:
# set the number of dictionary words 
# 50 for the small dataset
# 20,000 for the large dataset
numWords = 50

In [6]:
# load up the dataset 
# "20_news_same_line_random_sample.txt" for small dataset 
# "s3://s21risamyersbucket/lab/20_news_same_line.txt" for entire large dataset
corpus = sc.textFile ("20_news_same_line_random_sample.txt")

# each entry in validLines will be a line from the text file
validLines = corpus.filter(lambda x : 'id=' in x)

# now we transform it into a bunch of (docID, text) pairs
keyAndText = validLines.map(lambda x : (x[x.index('id="') + 4 : x.index('" url=')], x[x.index('"> ') + 3:x.index(' </doc>')]))

# now we split the text in each (docID, text) pair into a list of words
# after this, we have a data set with (docID, ["word1", "word2", "word3", ...])
# we have a bit of fancy regular expression stuff here to make sure that we do not
# die on some of the documents                       
regex = re.compile('[^a-zA-Z]')  
keyAndListOfWords = keyAndText.map(lambda x : (str(x[0]), regex.sub(' ', x[1]).lower().split()))

# now get the top 20,000 words... first change (docID, ["word1", "word2", "word3", ...])
# to ("word1", 1) ("word2", 1)...
allWords = keyAndListOfWords.flatMap(lambda x: ((j, 1) for j in x[1]))

# now, count all of the words, giving us ("word1", 1433), ("word2", 3423423), etc.
allCounts = allWords.reduceByKey (lambda a, b: a + b)

# and get the top numWords (50 for small dataset, 20K for large dataset) frequent words in a local array
topWords = allCounts.top (numWords, lambda x : x[1])

# and we'll create an RDD that has a bunch of (word, rank) pairs
# start by creating an RDD that has the number 0 up to numWords (50 for small dataset, 20K for large dataset) 
# numWords is the number of words that will be in our dictionary
numWordsRDD = sc.parallelize(range(numWords))

# now, we transform (0), (1), (2), ... to ("mostcommonword", 0) ("nextmostcommon", 1), ...
# the number will be the spot in the dictionary used to tell us where the word is located
refDict = numWordsRDD.map(lambda x:(topWords[x][0],x))

In [7]:
refDict.take(10)

[('the', 0),
 ('to', 1),
 ('of', 2),
 ('a', 3),
 ('and', 4),
 ('i', 5),
 ('in', 6),
 ('is', 7),
 ('that', 8),
 ('it', 9)]

Now, your task is to write Spark code, using RDDs, to create "bag of words" features based on the words in the reference dictionary, `refDict`.  

You need to create a new RDD, named `bag_of_words`. Each element of this RDD corresponds to one document, and is a key-value pair. Specifically, the key is the document identifier `id` (e.g. `20_newsgroups/comp.graphics/37261`) and the value is a `numpy` array with `numWords` (50 for the small dataset, 20K for the large dataset) entries, where the *i*th entry in the array is the number of times that the *i*th word in the `refDict` (created in the first part) appears in the document. This array corresponds to the "bag of words" features for each document.  

Once you created this `bag_of_words` RDD, print out the result arrays for these documents:
* `20_newsgroups/soc.religion.christian/21626`
* `20_newsgroups/talk.politics.misc/179019`
* `20_newsgroups/rec.autos/103167`

Since each array is going to be huge, with a lot of zeros, print out only the non-zero entries in the array (that is, for an array `a`, print out `a[a.nonzero()`].

In [8]:
# Makes RDD of word, docID for every word in every docID
word_docID = keyAndListOfWords.flatMap(lambda line: [(j,line[0]) for j in line[1]])

# Joins refDict with test2 to output RDD of (word's refDict index, docID))
rDidx_docID = refDict.leftOuterJoin(word_docID).map(lambda line: line[1])

In [9]:
# Adds a 1 count after each (refDict_idx, docID)
rDidx_docID_1count = rDidx_docID.map(lambda line: ((line),1))

In [10]:
# Extracts docIDs and indexes from refDict to make sure every word is accounted for (even zero counts)
docIDs = keyAndListOfWords.map(lambda x: x[0])
index = refDict.map(lambda x: x[1])
numberMapping = docIDs.cartesian(index)
numberMapping1 = numberMapping.map(lambda x: ((x[1], x[0]), 0))

# Unions the all numbers mapping with the docID words RDD to get all words in refDict with each docID accounted for
new_initial_counts = numberMapping1.union(rDidx_docID_1count)

In [23]:
# Sums count of each (refDict_idx, docID)
rDidx_docID_counts = new_initial_counts.reduceByKey(lambda a,b: a + b)

# Switches order and unpacks columns to (docID, rDidx, counts)
docID_rDidx_counts = rDidx_docID_counts.map(lambda x: (x[0][1], x[0][0], x[1])).sortBy(lambda x: x[1])

In [26]:
# Groups everything by the docID to create docID: [array of word indexes and counts] 
bag_of_words = docID_rDidx_counts.groupBy(lambda line: line[0]).map(lambda x : (x[0], np.array([j[2] for j in x[1]])))

Print results by running the code cells below:

In [27]:
arr1_1 = np.array(bag_of_words.lookup("20_newsgroups/soc.religion.christian/21626"))
arr1_1[arr1_1.nonzero()]

array([ 7,  2, 10,  4,  4,  5,  1,  6,  7,  8,  1,  1,  1,  3,  1,  2,  1,
        1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [28]:
arr1_2 = np.array(bag_of_words.lookup("20_newsgroups/talk.politics.misc/179019"))
arr1_2[arr1_2.nonzero()]

array([ 7, 23,  5, 17,  6,  5, 14, 10,  3, 20, 15,  4, 11,  1,  1,  4,  4,
        8,  4,  3,  2,  1,  2,  3, 10,  3,  1,  1,  1,  1,  1,  1,  2,  1,
        1,  1,  2,  2,  2,  3])

In [29]:
arr1_3 = np.array(bag_of_words.lookup("20_newsgroups/rec.autos/103167"))
arr1_3[arr1_3.nonzero()]

array([9, 1, 2, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1])

## (25 pts) Task 2 - compute the TF-IDF value for each document 

It is often difficult to classify documents accurately using raw count vectors (bag of words). Thus, the next task is
to write some more Spark RDD code that converts each of the count vectors to TF-IDF vectors. You need to create an RDD of key-value pairs, named `tfidf`, where the keys are document identifiers, and the values are the TF-IDF vector for that document. Again, we are only interested in the top `numWords` (50 for small dataset, 20K for large dataset) most common words.  

The ith entry in a TF-IDF vector corresponds to the ith word in the top `numWords` most common words dictionary `refDict`. Then, the ith entry in a TF-IDF vector for document $d$ in a corpus with $D$ documents is computed as:

$$ TF(i, d) \times IDF(i, D) $$

Where $TF(i, d)$ is: 

$$ \frac {\textrm{# of occurences of word $i$ in $d$}} {\textrm{Total # of words in $d$}} $$

Note that the “Total number of words” is not the number of distinct words. The “total number of words”
in “Today is a great day today” is six. 

And the $IDF(i,D)$ is:

$$ \log \frac {\textrm{# of documents in corpus D}} {(\textrm{# of documents having word $i$}) + 1} $$

Once you created this `tfidf` RDD, print out the **non-zero** array entries (TF-IDF vector) that you have created for these documents:

* `20_newsgroups/soc.religion.christian/21626`
* `20_newsgroups/talk.politics.misc/179019`
* `20_newsgroups/rec.autos/103167`

In [17]:
# TF
tf = bag_of_words.map(lambda x: (x[0], [j / np.sum(x[1]) for j in x[1]]))

In [51]:
# IDF 
# Gets number of documents
num_docs = bag_of_words.count()

distinctWords = rDidx_docID_1count.map(lambda x: x[0]).distinct()
groupedDistinct = distinctWords.groupByKey().map(lambda x : (x[0], np.array(list(x[1]))))
countWord = groupedDistinct.map(lambda x: (x[0], np.count_nonzero(x[1]) + 1)).sortBy(lambda x: x[0])

idfValue = countWord.map(lambda x: np.log(num_docs / x[1]))

idf = np.array(idfValue.collect())
tf_idf = tf.map(lambda x: (x[0], np.multiply(x[1], idf)))

Once you created your `tfidf` RDD, print out the **non-zero** result arrays for these documents,
* `20_newsgroups/soc.religion.christian/21626`
* `20_newsgroups/talk.politics.misc/179019`
* `20_newsgroups/rec.autos/103167`

by running the code cells below:

In [50]:
arr2_1 = tf_idf.filter(lambda x: x[0]=='20_newsgroups/soc.religion.christian/21626').values()
arr2_1 = arr2_1.collect()[0]
arr2_1[arr2_1.nonzero()]

array([ 6.10988634e-03,  3.40904641e-03,  2.67266121e-02,  6.99609887e-03,
        9.99334507e-03,  1.21776054e-02,  1.92857154e-03,  1.88627191e-02,
        2.88594755e-02,  3.39860926e-02, -1.29805238e-05,  6.18732723e-03,
        6.44074041e-03,  2.48828180e-02,  7.55332215e-03,  1.30097479e-02,
        6.67746136e-03,  7.50693992e-03,  1.69851030e-02,  8.46760048e-03,
       -1.29805238e-05, -1.29805238e-05,  5.20522259e-05,  9.45115098e-03,
        1.08403993e-02,  2.03446507e-03,  1.21291645e-02,  1.40871349e-02,
        1.29475147e-02])

In [34]:
arr2_2 = tf_idf.filter(lambda x: x[0]=='20_newsgroups/talk.politics.misc/179019').values()
arr2_2 = arr2_2.collect()[0]
arr2_2[arr2_2.nonzero()]

array([ 2.28379247e-03,  1.46539349e-02,  4.99502216e-03,  1.11139483e-02,
        5.60306483e-03,  3.60436913e-03,  1.64514653e-02,  1.54103996e-02,
        4.76382731e-03,  5.23852517e-02,  2.31155993e-02,  1.01468132e-02,
        3.04865293e-02, -4.85194336e-06,  2.31273882e-03,  9.62984488e-03,
        1.24011455e-02,  2.14906889e-02,  1.12933166e-02,  7.29430038e-03,
        4.99188859e-03,  3.74849703e-03,  5.61198421e-03,  6.96168642e-03,
        3.16507397e-02,  1.29489621e-02, -4.85194336e-06, -4.85194336e-06,
        1.94564146e-05,  2.90048565e-04,  1.62172384e-03,  4.57091524e-03,
        8.10398781e-03,  7.60455389e-04,  4.53371683e-03,  4.15371898e-03,
        8.79942137e-03,  1.05311591e-02,  5.78660650e-03,  1.31272153e-02])

In [20]:
arr2_3 = tf_idf.filter(lambda x: x[0]=='20_newsgroups/rec.autos/103167').values()
arr2_3 = arr2_3.collect()[0]
arr2_3[arr2_3.nonzero()]

array([ 1.18603676e-02,  2.57349582e-03,  7.54399579e-03,  1.10314779e-02,
        5.82352974e-03,  4.74650120e-03,  6.22459276e-03,  6.22459276e-03,
       -1.95980457e-05,  9.34165092e-03,  1.25227254e-02,  1.14040354e-02,
        9.82108417e-03,  2.01633147e-02,  1.28220876e-02,  5.03333006e-02,
       -1.95980457e-05, -1.95980457e-05,  7.85886549e-05,  1.17156871e-03,
        6.55049239e-03,  3.07164333e-03,  1.73393664e-02,  1.89722358e-02,
        5.10527487e-02,  3.06945299e-02,  2.74189597e-02,  1.05421473e-01,
        3.08830683e-02,  5.90226626e-02,  3.17546715e-02,  3.79988623e-02,
        3.53296040e-02,  2.94232060e-02])

## (30 pts) Task 3 - build a kNN classifier 

Task 3 is to build a kNN classifier, as a Python function named `predictLabel` in the cell below. This function will take as input a text string (`test_doc`) and a number *k*, and then output the name of one of the 20 newsgroups. This name is the news group that the classifier thinks that the text string is “closest” to. It is computed using the classical kNN algorithm. 

Your function first needs to convert the input string into all **lower case** words, and then compute a TF-IDF vector corresponding to the words in `refDict` created in the first task. Recall that the words in `refDict` is our reference words to compute "TF-IDF" features. In task 2, we already computed TF-IDF values of these words for our training corpus. In this task, you need to compute the TF-IDF values of these words for the input text string `test_doc`. For that, you need to compute term frequency of these words in the `test_doc`.

Since the IDF measure of a word only depends on the training corpus, and this measure is already calculated for `refDict` words in task 2, you don't need to re-calculate IDF for the `test_doc` and can re-use what you have.  

Next, your function needs to find the *k* documents in the corpus that are “closest” to the `test_doc` (where distance is computed using the L2 norm between the TF-IDF feature vectors), and returns the newsgroup label that is most frequent in those top *k*. Ties go to the label with the closest corpus document. If there are multiple labels the same distance from the document, choose randomly. If you have the neighbors in a list, choosing the first of the tied values is fine, since there was some stochasticity in populating the list.

Note that the numpy function `np.linalg.norm` computes the L2 norm of a vector. You can call it with the difference of the training document TF-IDF vectors and the test document TF-IDF vector. See the documentation here:[np.linalg.norm](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html)

Keep in mind that you should be leveraging Spark. Do NOT loop over the training documents. Instead, use your RDD with the TF-IDF vectors for each document.

Once you have implemented your function, run it on the following 8 test cases. Each is an excerpt from a Wikipedia article, chosen to match one of the 20 newsgroups. By reading each test document, you can guess which of the 20 newsgroups is the most relevent topic, and you can compare that with what your prediction function returns. The result you get from the small dataset might not be so accurate, due to the small training corpus. But, once you run it on the entire dataset in S3, you should see reasonable results.  

In [52]:
# k is the number of neighbors to consider
# test_doc is the text to compare 
def predictLabel (k, test_doc):
    # your code here
    regex = re.compile('[^a-zA-Z]')  
    listOfDocWords = sc.parallelize(regex.sub(' ', test_doc).lower().split())
    docWords1 = listOfDocWords.map(lambda x: (x, 1))

    # Subsets only words in refDict
    rD_docWords = refDict.join(docWords1)
    rD_docWords2 = rD_docWords.map(lambda x: ((x[0], x[1][0]), x[1][1]))

    # Counts of words in test_doc in form (wordIdx, counts)
    docWords_counts = rD_docWords2.map(lambda x: (x[0][1], x[1]))

    # RDD of all indexes in refDict to account for all words in refDict
    index2 = refDict.map(lambda x: (x[1], 0))

    rD_docWords_counts = docWords_counts.union(index2).reduceByKey(lambda a,b: a+b)
    countsOnly = rD_docWords_counts.sortByKey().map(lambda x: x[1])
    countList = countsOnly.collect()

    # Compute TF for test doc
    tfTest = countsOnly.map(lambda x: x / sum(countList))
    tf_idfTest = np.array(tfTest.collect()) * idf

    # k neighbors
    tfidf_top_norms = sc.parallelize(tf_idf.map(lambda x: (x[0], np.linalg.norm(x[1] - tf_idfTest))).sortBy(lambda x: x[1]).take(k))
    #tfidf_top_norms.collect()
    # Gets the top categories from the docIDs, with the L2 norms
    top_categories_norms = tfidf_top_norms.map(lambda x: (x[0].split('/')[1], x[1]))

    # RDD of (category, [l2 norms]), sorted by most l2 norms
    top_categories = top_categories_norms.groupBy(lambda x: x[0]).map(lambda x : (x[0], [j[1] for j in x[1]], len(x[1]))).sortBy(lambda x: x[2], ascending=False)

    # Eliminating ties for top category
    if top_categories.take(1)[0][2] == top_categories.take(2)[1][2]:
      tied_len = top_categories.take(1)[0][2]
      new_top_cat = top_categories.filter(lambda x: x[2] == tied_len).sortBy(lambda x: x[1][0])
      top = new_top_cat.take(1)[0][0]
    else:
      top = top_categories.take(1)[0][0]


    return top

#### Test cases

Run your predictLabel function on the 8 test cases below.

In [53]:
print(predictLabel (10, 'Graphics are pictures and movies created using computers – usually referring to image data created by a computer specifically with help from specialized graphical hardware and software. It is a vast and recent area in computer science. The phrase was coined by computer graphics researchers Verne Hudson and William Fetter of Boeing in 1960. It is often abbreviated as CG, though sometimes erroneously referred to as CGI. Important topics in computer graphics include user interface design, sprite graphics, vector graphics, 3D modeling, shaders, GPU design, implicit surface visualization with ray tracing, and computer vision, among others. The overall methodology depends heavily on the underlying sciences of geometry, optics, and physics. Computer graphics is responsible for displaying art and image data effectively and meaningfully to the user, and processing image data received from the physical world. The interaction and understanding of computers and interpretation of data has been made easier because of computer graphics. Computer graphic development has had a significant impact on many types of media and has revolutionized animation, movies, advertising, video games, and graphic design generally.'))

talk.politics.mideast


In [54]:
print(predictLabel (10, 'A deity is a concept conceived in diverse ways in various cultures, typically as a natural or supernatural being considered divine or sacred. Monotheistic religions accept only one Deity (predominantly referred to as God), polytheistic religions accept and worship multiple deities, henotheistic religions accept one supreme deity without denying other deities considering them as equivalent aspects of the same divine principle, while several non-theistic religions deny any supreme eternal creator deity but accept a pantheon of deities which live, die and are reborn just like any other being. A male deity is a god, while a female deity is a goddess. The Oxford reference defines deity as a god or goddess (in a polytheistic religion), or anything revered as divine. C. Scott Littleton defines a deity as a being with powers greater than those of ordinary humans, but who interacts with humans, positively or negatively, in ways that carry humans to new levels of consciousness beyond the grounded preoccupations of ordinary life.'))

talk.politics.guns


In [39]:
print(predictLabel (10, 'Egypt, officially the Arab Republic of Egypt, is a transcontinental country spanning the northeast corner of Africa and southwest corner of Asia by a land bridge formed by the Sinai Peninsula. Egypt is a Mediterranean country bordered by the Gaza Strip and Israel to the northeast, the Gulf of Aqaba to the east, the Red Sea to the east and south, Sudan to the south, and Libya to the west. Across the Gulf of Aqaba lies Jordan, and across from the Sinai Peninsula lies Saudi Arabia, although Jordan and Saudi Arabia do not share a land border with Egypt. It is the worlds only contiguous Eurafrasian nation. Egypt has among the longest histories of any modern country, emerging as one of the worlds first nation states in the tenth millennium BC. Considered a cradle of civilisation, Ancient Egypt experienced some of the earliest developments of writing, agriculture, urbanisation, organised religion and central government. Iconic monuments such as the Giza Necropolis and its Great Sphinx, as well the ruins of Memphis, Thebes, Karnak, and the Valley of the Kings, reflect this legacy and remain a significant focus of archaeological study and popular interest worldwide. Egypts rich cultural heritage is an integral part of its national identity, which has endured, and at times assimilated, various foreign influences, including Greek, Persian, Roman, Arab, Ottoman, and European. One of the earliest centers of Christianity, Egypt was Islamised in the seventh century and remains a predominantly Muslim country, albeit with a significant Christian minority.'))

talk.politics.mideast


In [40]:
print(predictLabel (10, 'The term atheism originated from the Greek atheos, meaning without god(s), used as a pejorative term applied to those thought to reject the gods worshiped by the larger society. With the spread of freethought, skeptical inquiry, and subsequent increase in criticism of religion, application of the term narrowed in scope. The first individuals to identify themselves using the word atheist lived in the 18th century during the Age of Enlightenment. The French Revolution, noted for its unprecedented atheism, witnessed the first major political movement in history to advocate for the supremacy of human reason. Arguments for atheism range from the philosophical to social and historical approaches. Rationales for not believing in deities include arguments that there is a lack of empirical evidence; the problem of evil; the argument from inconsistent revelations; the rejection of concepts that cannot be falsified; and the argument from nonbelief. Although some atheists have adopted secular philosophies (eg. humanism and skepticism), there is no one ideology or set of behaviors to which all atheists adhere.'))

talk.politics.mideast


In [41]:
print(predictLabel (10, 'President Dwight D. Eisenhower established NASA in 1958 with a distinctly civilian (rather than military) orientation encouraging peaceful applications in space science. The National Aeronautics and Space Act was passed on July 29, 1958, disestablishing NASAs predecessor, the National Advisory Committee for Aeronautics (NACA). The new agency became operational on October 1, 1958. Since that time, most US space exploration efforts have been led by NASA, including the Apollo moon-landing missions, the Skylab space station, and later the Space Shuttle. Currently, NASA is supporting the International Space Station and is overseeing the development of the Orion Multi-Purpose Crew Vehicle, the Space Launch System and Commercial Crew vehicles. The agency is also responsible for the Launch Services Program (LSP) which provides oversight of launch operations and countdown management for unmanned NASA launches.'))

talk.politics.mideast


In [42]:
print(predictLabel (10, 'The transistor is the fundamental building block of modern electronic devices, and is ubiquitous in modern electronic systems. First conceived by Julius Lilienfeld in 1926 and practically implemented in 1947 by American physicists John Bardeen, Walter Brattain, and William Shockley, the transistor revolutionized the field of electronics, and paved the way for smaller and cheaper radios, calculators, and computers, among other things. The transistor is on the list of IEEE milestones in electronics, and Bardeen, Brattain, and Shockley shared the 1956 Nobel Prize in Physics for their achievement.'))

talk.politics.mideast


In [43]:
print(predictLabel (10, 'The Colt Single Action Army which is also known as the Single Action Army, SAA, Model P, Peacemaker, M1873, and Colt .45 is a single-action revolver with a revolving cylinder holding six metallic cartridges. It was designed for the U.S. government service revolver trials of 1872 by Colts Patent Firearms Manufacturing Company – todays Colts Manufacturing Company – and was adopted as the standard military service revolver until 1892. The Colt SAA has been offered in over 30 different calibers and various barrel lengths. Its overall appearance has remained consistent since 1873. Colt has discontinued its production twice, but brought it back due to popular demand. The revolver was popular with ranchers, lawmen, and outlaws alike, but as of the early 21st century, models are mostly bought by collectors and re-enactors. Its design has influenced the production of numerous other models from other companies.'))

talk.politics.mideast


In [44]:
print(predictLabel (10, 'Howe was recruited by the Red Wings and made his NHL debut in 1946. He led the league in scoring each year from 1950 to 1954, then again in 1957 and 1963. He ranked among the top ten in league scoring for 21 consecutive years and set a league record for points in a season (95) in 1953. He won the Stanley Cup with the Red Wings four times, won six Hart Trophies as the leagues most valuable player, and won six Art Ross Trophies as the leading scorer. Howe retired in 1971 and was inducted into the Hockey Hall of Fame the next year. However, he came back two years later to join his sons Mark and Marty on the Houston Aeros of the WHA. Although in his mid-40s, he scored over 100 points twice in six years. He made a brief return to the NHL in 1979–80, playing one season with the Hartford Whalers, then retired at the age of 52. His involvement with the WHA was central to their brief pre-NHL merger success and forced the NHL to expand their recruitment to European talent and to expand to new markets.'))

talk.politics.mideast


## (15 pts) Task 4 - run on the entire dataset in EMR cluster 

For the last part of this homework, you need to run your Spark code for tasks 1 through 3, on the entire dataset stored in S3, using an AWS EMR cluster. 

Follow the instructions from `Lab - Spark Intro (AWS)` to create and connect to an EMR cluster in AWS and run Spark programs there. You can put your code for each task in a Python `.py` file and submit them as jobs in the batch mode and get the final result back. To troubleshoot, you can run your code line by line, in an interactive mode to debug your program.       

The entire dataset exists in this S3 URI: `s3://s21risamyersbucket/lab/20_news_same_line.txt`

Repeat tasks 1 through 3 on the entire dataset in your EMR cluster, and print your results in the markdown cells below (keep the results from the small subset above). 

**Repeat task 1 on the entire dataset in your EMR cluster - print out the non-zero array entries (bag of words) that you have created for documents:**

* `20_newsgroups/soc.religion.christian/21626`
* `20_newsgroups/talk.politics.misc/179019`
* `20_newsgroups/rec.autos/103167`

array([ 2,  1,  2,  1,  2,  2,  1,  1,  3,  1,  1,  1,  5,  3,  1,  2,  1,
        1,  1,  1,  1,  2,  3,  1,  1,  7,  2, 10,  4,  4,  5,  1,  6,  7,
        8,  1,  1,  1,  1,  3,  2,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,
        1,  3,  1,  1,  1,  1,  1,  4,  1,  1,  1,  1,  2,  2,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,
        1,  2,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  1,  1,  3,  1,  1,
        1,  1,  1,  1,  1,  1,  2,  1,  2,  1,  1,  2,  1,  1,  1,  1,  2])

array([ 1,  1,  1,  1,  1,  2,  2,  2,  1,  1,  1,  2,  2,  1,  5,  1,  2,
        1,  4,  1,  2,  1,  1,  2,  1,  4,  3,  1,  1,  1,  7, 23,  5, 17,
        6,  5, 14, 10,  3, 15, 20,  4,  1, 11,  1,  4,  8,  4,  4,  3,  2,
        1,  2,  3, 10,  3,  1,  1,  1,  1,  1,  1,  2,  1,  1,  1,  1,  2,
        2,  3,  5,  2,  2,  1,  1,  2,  1,  1,  8,  1,  2,  1,  3,  1,  1,
        2,  1,  2,  1,  1,  2,  2,  3,  1,  1,  1,  1,  2,  1,  1, 11,  3,
        1,  1,  1,  1,  1,  2,  1,  2,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  2,  2,  1,  1,  1,  1,  1,  1,  2,  1,  1,  2,  1,  1,  6,  1,
        3,  1,  3,  1,  1,  1,  3,  3,  2,  1,  1,  3, 11,  2,  1,  1,  1,
        1, 11,  1,  1,  1,  1,  1,  1,  1,  2,  3,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1])

array([1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 9, 1, 2, 3, 2, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 2, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1])


**Repeat task 2 on the entire dataset in your EMR cluster - print out the non-zero array entries (TF-IDF) that you have created for documents:**

* `20_newsgroups/soc.religion.christian/21626`
* `20_newsgroups/talk.politics.misc/179019`
* `20_newsgroups/rec.autos/103167`

array([ 2.55502668e-03, 1.29010463e-03, 9.25293813e-03, 2.57906645e-03, 3.67581489e-03, 4.54955151e-03, 8.34284642e-04, 8.07815673e-03, 1.17952717e-02, 1.33848412e-02, -2.51287692e-07, 2.51413480e-03, 2.57081759e-03, 2.99842959e-03, 9.55455241e-03, 5.23511872e-03, 2.92728501e-03, 3.31244031e-03, 3.30807000e-03, 6.65602007e-03, -2.51287692e-07, 1.43442099e-05, -2.51287692e-07, 3.95742539e-03, 4.36932519e-03, 8.16219817e-04, 4.73739398e-03, 1.36238925e-02, 4.97312923e-03, 5.69998357e-03, 4.91532217e-03, 5.00500475e-03, 5.36527949e-03, 2.04797150e-02, 6.31505670e-03, 5.32306353e-03, 5.83217927e-03, 6.42034550e-03, 1.27544993e-02, 2.49103093e-02, 6.45745150e-03, 1.45971251e-02, 1.23867036e-02, 7.43064488e-03, 7.57378230e-03, 7.21696122e-03, 1.34638831e-02, 8.62610429e-03, 8.10575352e-03, 8.16792605e-03, 1.86790398e-02, 8.74068636e-03, 1.30526508e-02, 8.97350148e-03, 9.22392636e-03, 1.69735918e-02, 1.05831471e-02, 1.31790972e-02, 1.21190556e-02, 1.16027955e-02, 1.21443429e-02, 2.39507838e-02, 1.20494737e-02, 1.18383711e-02, 1.29856034e-02, 1.30594050e-02, 1.22669477e-02, 2.89226053e-02, 1.47084898e-02, 1.33659526e-02, 1.45468963e-02, 1.39884518e-02, 1.39965896e-02, 1.50031755e-02, 4.79141291e-02, 1.42900498e-02, 1.55882136e-02, 1.55659045e-02, 1.71335864e-02, 1.80300479e-02, 1.85162685e-02, 1.92825601e-02, 1.69105413e-02, 3.43128906e-02, 1.87519444e-02, 3.47818832e-02, 1.65527050e-02, 1.85868331e-02, 4.07728519e-02, 1.89444196e-02, 1.94005262e-02, 1.94364629e-02, 2.27077511e-02, 4.23075920e-02, 5.23818056e-02, 2.52670363e-02, 2.53443474e-02, 2.84885982e-02, 9.96105629e-02, 7.10565246e-02, 3.33931709e-02, 7.16658173e-02, 3.52410341e-02, 6.80185259e-02, 3.65039207e-02, 3.87241857e-02, 3.61572228e-02, 4.27992120e-02, 3.99870723e-02, 3.58329086e-02, 8.33557752e-02, 1.07498726e-01, 3.58329086e-02, 3.58329086e-02, 1.82519603e-01, 1.14584208e-01, 3.99870723e-02, 7.86321206e-02, 3.99870723e-02])


array([0.00881331, 0.01370039, 0.01044009, 0.02205 , 0.03316673, 0.01155318, 0.01113388, 0.01240744, 0.02263282, 0.08807844, 0.28802224, 0.06232167, 0.26024498, 0.06821326, 0.06058964, 0.16930716, 0.11775053, 0.03551871, 0.18176892, 0.3298152 , 0.04985734, 0.01246433, 0.1485993 , 0.01224492, 0.04847171, 0.11813273, 0.04710021, 0.05122377, 0.03585418, 0.02481488, 0.01235195, 0.02413815, 0.03987205, 0.11651323, 0.03765741, 0.01188363, 0.01186149, 0.01375089, 0.01167148, 0.01214276, 0.01402197, 0.02541502, 0.01177505, 0.01240744, 0.01177505, 0.01157247, 0.02701806, 0.02359271, 0.04140767, 0.07654264, 0.0234661 , 0.02418674, 0.01280594, 0.01171234, 0.02330265, 0.01216787, 0.01165132, 0.09948566, 0.01337549, 0.02385716, 0.01283974, 0.0368934 , 0.01204504, 0.01235195, 0.02504541, 0.01280594, 0.02522642, 0.01211793, 0.01273985, 0.02399546, 0.02683826, 0.0365036 , 0.01298023, 0.01186149, 0.01171234, 0.01227121, 0.02454241, 0.01301677, 0.01175395, 0.14199775, 0.04299086, 0.01224492, 0.01175395, 0.01270751, 0.01232471, 0.01287405, 0.02516527, 0.01261321, 0.02381199, 0.01204504, 0.01204504, 0.01360267, 0.01380256, 0.01252271, 0.01167148, 0.01280594, 0.01261321, 0.01195139, 0.01309169, 0.02516527, 0.0262602 , 0.01255247, 0.01277265, 0.01179635, 0.01232471, 0.01195139, 0.01232471, 0.02804394, 0.01186149, 0.01370039, 0.02385716, 0.01264421, 0.01181786, 0.07256021, 0.01216787, 0.0393903 , 0.01197444, 0.03538906, 0.01309169, 0.01211793, 0.01209337, 0.0415663 , 0.03592333, 0.02666537, 0.01232471, 0.01270751, 0.03802691, 0.13840898, 0.0262602 , 0.01216787, 0.01221894, 0.01309169, 0.01337549, 0.13943199, 0.01290889, 0.01301677, 0.01258263, 0.01267564, 0.01333269, 0.01301677, 0.06209083, 0.0393903 , 0.01329068, 0.01355536, 0.01468773, 0.02701806, 0.01632423, 0.01426514, 0.01822827, 0.01476659, 0.01520864, 0.01667063, 0.01588792, 0.01552271, 0.01530853, 0.01575947, 0.03233837, 0.03334126, 0.01863798, 0.01575947, 0.01602415, 0.03151895, 0.03827885, 0.01575947, 0.08012077, 0.01616919, 0.03373232, 0.01588792, 0.06409662, 0.01632423, 0.03177584, 0.01667063, 0.01588792, 0.01822827, 0.01602415, 0.01602415, 0.01649076, 0.01686616])

array([0.07647283, 0.05482856, 0.05859618, 0.05922118, 0.18408538, 0.06262573, 0.06240402, 0.06116542, 0.06738187, 0.15295734, 0.15671655, 0.07351287, 0.07174122, 0.07488949, 0.07288788, 0.66760398, 0.07351287, 0.15295734, 0.21689585, 0.15888893, 0.07229862, 0.07288788, 0.07417822, 0.07944446, 0.07647867, 0.07565349, 0.07288788, 0.08065872, 0.15295734, 0.21689585, 0.07835827, 0.07565349, 0.07565349, 0.07647867, 0.07647867, 0.08065872, 0.07835827, 0.08362452, 0.07647867, 0.08203533, 0.07488949, 0.07737569, 0.07351287, 0.14835644, 0.07488949, 0.14977897, 0.07288788, 0.07351287, 0.08362452, 0.22696046, 0.07835827, 0.07835827, 0.07417822, 0.07565349, 0.07417822, 0.07565349, 0.09077036, 0.22253466, 0.07835827, 0.07737569, 0.07417822, 0.07351287, 0.07351287, 0.07417822, 0.07417822, 0.08362452, 0.07565349, 0.07944446, 0.08550412, 0.07417822, 0.07835827])

**Repeat task 3 on the entire dataset in your EMR cluster - print out the predicted label for each of the below test document:**

In [ ]:
print(predictLabel (10, 'Graphics are pictures and movies created using computers – usually referring to image data created by a computer specifically with help from specialized graphical hardware and software. It is a vast and recent area in computer science. The phrase was coined by computer graphics researchers Verne Hudson and William Fetter of Boeing in 1960. It is often abbreviated as CG, though sometimes erroneously referred to as CGI. Important topics in computer graphics include user interface design, sprite graphics, vector graphics, 3D modeling, shaders, GPU design, implicit surface visualization with ray tracing, and computer vision, among others. The overall methodology depends heavily on the underlying sciences of geometry, optics, and physics. Computer graphics is responsible for displaying art and image data effectively and meaningfully to the user, and processing image data received from the physical world. The interaction and understanding of computers and interpretation of data has been made easier because of computer graphics. Computer graphic development has had a significant impact on many types of media and has revolutionized animation, movies, advertising, video games, and graphic design generally.'))
print(predictLabel (10, 'A deity is a concept conceived in diverse ways in various cultures, typically as a natural or supernatural being considered divine or sacred. Monotheistic religions accept only one Deity (predominantly referred to as God), polytheistic religions accept and worship multiple deities, henotheistic religions accept one supreme deity without denying other deities considering them as equivalent aspects of the same divine principle, while several non-theistic religions deny any supreme eternal creator deity but accept a pantheon of deities which live, die and are reborn just like any other being. A male deity is a god, while a female deity is a goddess. The Oxford reference defines deity as a god or goddess (in a polytheistic religion), or anything revered as divine. C. Scott Littleton defines a deity as a being with powers greater than those of ordinary humans, but who interacts with humans, positively or negatively, in ways that carry humans to new levels of consciousness beyond the grounded preoccupations of ordinary life.'))
print(predictLabel (10, 'Egypt, officially the Arab Republic of Egypt, is a transcontinental country spanning the northeast corner of Africa and southwest corner of Asia by a land bridge formed by the Sinai Peninsula. Egypt is a Mediterranean country bordered by the Gaza Strip and Israel to the northeast, the Gulf of Aqaba to the east, the Red Sea to the east and south, Sudan to the south, and Libya to the west. Across the Gulf of Aqaba lies Jordan, and across from the Sinai Peninsula lies Saudi Arabia, although Jordan and Saudi Arabia do not share a land border with Egypt. It is the worlds only contiguous Eurafrasian nation. Egypt has among the longest histories of any modern country, emerging as one of the worlds first nation states in the tenth millennium BC. Considered a cradle of civilisation, Ancient Egypt experienced some of the earliest developments of writing, agriculture, urbanisation, organised religion and central government. Iconic monuments such as the Giza Necropolis and its Great Sphinx, as well the ruins of Memphis, Thebes, Karnak, and the Valley of the Kings, reflect this legacy and remain a significant focus of archaeological study and popular interest worldwide. Egypts rich cultural heritage is an integral part of its national identity, which has endured, and at times assimilated, various foreign influences, including Greek, Persian, Roman, Arab, Ottoman, and European. One of the earliest centers of Christianity, Egypt was Islamised in the seventh century and remains a predominantly Muslim country, albeit with a significant Christian minority.'))
print(predictLabel (10, 'The term atheism originated from the Greek atheos, meaning without god(s), used as a pejorative term applied to those thought to reject the gods worshiped by the larger society. With the spread of freethought, skeptical inquiry, and subsequent increase in criticism of religion, application of the term narrowed in scope. The first individuals to identify themselves using the word atheist lived in the 18th century during the Age of Enlightenment. The French Revolution, noted for its unprecedented atheism, witnessed the first major political movement in history to advocate for the supremacy of human reason. Arguments for atheism range from the philosophical to social and historical approaches. Rationales for not believing in deities include arguments that there is a lack of empirical evidence; the problem of evil; the argument from inconsistent revelations; the rejection of concepts that cannot be falsified; and the argument from nonbelief. Although some atheists have adopted secular philosophies (eg. humanism and skepticism), there is no one ideology or set of behaviors to which all atheists adhere.'))
print(predictLabel (10, 'President Dwight D. Eisenhower established NASA in 1958 with a distinctly civilian (rather than military) orientation encouraging peaceful applications in space science. The National Aeronautics and Space Act was passed on July 29, 1958, disestablishing NASAs predecessor, the National Advisory Committee for Aeronautics (NACA). The new agency became operational on October 1, 1958. Since that time, most US space exploration efforts have been led by NASA, including the Apollo moon-landing missions, the Skylab space station, and later the Space Shuttle. Currently, NASA is supporting the International Space Station and is overseeing the development of the Orion Multi-Purpose Crew Vehicle, the Space Launch System and Commercial Crew vehicles. The agency is also responsible for the Launch Services Program (LSP) which provides oversight of launch operations and countdown management for unmanned NASA launches.'))
print(predictLabel (10, 'The transistor is the fundamental building block of modern electronic devices, and is ubiquitous in modern electronic systems. First conceived by Julius Lilienfeld in 1926 and practically implemented in 1947 by American physicists John Bardeen, Walter Brattain, and William Shockley, the transistor revolutionized the field of electronics, and paved the way for smaller and cheaper radios, calculators, and computers, among other things. The transistor is on the list of IEEE milestones in electronics, and Bardeen, Brattain, and Shockley shared the 1956 Nobel Prize in Physics for their achievement.'))
print(predictLabel (10, 'The Colt Single Action Army which is also known as the Single Action Army, SAA, Model P, Peacemaker, M1873, and Colt .45 is a single-action revolver with a revolving cylinder holding six metallic cartridges. It was designed for the U.S. government service revolver trials of 1872 by Colts Patent Firearms Manufacturing Company – todays Colts Manufacturing Company – and was adopted as the standard military service revolver until 1892. The Colt SAA has been offered in over 30 different calibers and various barrel lengths. Its overall appearance has remained consistent since 1873. Colt has discontinued its production twice, but brought it back due to popular demand. The revolver was popular with ranchers, lawmen, and outlaws alike, but as of the early 21st century, models are mostly bought by collectors and re-enactors. Its design has influenced the production of numerous other models from other companies.'))
print(predictLabel (10, 'Howe was recruited by the Red Wings and made his NHL debut in 1946. He led the league in scoring each year from 1950 to 1954, then again in 1957 and 1963. He ranked among the top ten in league scoring for 21 consecutive years and set a league record for points in a season (95) in 1953. He won the Stanley Cup with the Red Wings four times, won six Hart Trophies as the leagues most valuable player, and won six Art Ross Trophies as the leading scorer. Howe retired in 1971 and was inducted into the Hockey Hall of Fame the next year. However, he came back two years later to join his sons Mark and Marty on the Houston Aeros of the WHA. Although in his mid-40s, he scored over 100 points twice in six years. He made a brief return to the NHL in 1979–80, playing one season with the Hartford Whalers, then retired at the age of 52. His involvement with the WHA was central to their brief pre-NHL merger success and forced the NHL to expand their recruitment to European talent and to expand to new markets.'))

comp.graphics \\
soc.religion.christian \\
talk.politics.mideast \\
alt.atheism \\
sci.space \\
sci.electronics \\
talk.politics.guns \\
rec.sport.hockey

# Don't forget to stop your Spark context when you are done!

In [55]:
sc.stop()

Copy all the predicted labels in this markdown cell ...


### Copyright ©2021 Christopher M Jermaine (cmj4@rice.edu), Risa B Myers  (rbm2@rice.edu), Marmar Orooji (marmar.orooji@rice.edu)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.